<a href="https://colab.research.google.com/github/Eboselethefirst/Mini_projects/blob/main/ETL_With_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark

In [2]:
def warn(*arbgs,**kwargs):
  pass
import warnings as wng
wng.warn = warn
wng.filterwarnings("ignore")


In [3]:
import findspark as fs
fs.init()

from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("ETL using Spark").getOrCreate()

In [5]:
data = [("student1",64,90),
        ("student2",59,100),
        ("student3",69,95),
        ("",70,110),
        ("student5",60,80),
        ("student3",69,95),
        ("student6",62,85),
        ("student7",65,80),
        ("student7",65,80)]

In [6]:
df = spark.createDataFrame(data, ["student","height inches","weight pounds"])

In [7]:
df.show()

+--------+-------------+-------------+
| student|height inches|weight pounds|
+--------+-------------+-------------+
|student1|           64|           90|
|student2|           59|          100|
|student3|           69|           95|
|        |           70|          110|
|student5|           60|           80|
|student3|           69|           95|
|student6|           62|           85|
|student7|           65|           80|
|student7|           65|           80|
+--------+-------------+-------------+



In [8]:
df.write.mode("overwrite").csv("student-hw.csv", header= True)

In [9]:
!ls

sample_data  student-hw.csv


In [10]:
#To verify is the file was saved
df1 = spark.read.csv("student-hw.csv",header=True,inferSchema=True)
df1.show()

+--------+-------------+-------------+
| student|height inches|weight pounds|
+--------+-------------+-------------+
|student5|           60|           80|
|student3|           69|           95|
|student6|           62|           85|
|student7|           65|           80|
|student7|           65|           80|
|student1|           64|           90|
|student2|           59|          100|
|student3|           69|           95|
|    NULL|           70|          110|
+--------+-------------+-------------+



In [11]:
df1.count()

9

In [13]:
#dropduplicates
df = df1.dropDuplicates()

In [14]:
df.count()

7

In [18]:
df =df.dropna()

In [19]:
df.show()

+--------+-------------+-------------+
| student|height inches|weight pounds|
+--------+-------------+-------------+
|student6|           62|           85|
|student3|           69|           95|
|student7|           65|           80|
|student5|           60|           80|
|student2|           59|          100|
|student1|           64|           90|
+--------+-------------+-------------+



In [20]:
df.write.mode("overwrite").parquet("student-hw.parquet")

In [21]:
!ls

sample_data  student-hw.csv  student-hw.parquet


In [26]:
!!ls -l student-hw.parquet

['total 4',
 '-rw-r--r-- 1 root root 1114 Jan 21 17:11 part-00000-5d4beb0e-7422-4e8e-a20f-73f3d9f6c5df-c000.snappy.parquet',
 '-rw-r--r-- 1 root root    0 Jan 21 17:11 _SUCCESS']

from the previous command the number of files created for parquet are 4 seperate files and they need to be condensed


In [36]:
df=df.repartition(1)

In [37]:
df.write.mode("overwrite").parquet("student-hw-omo.parquet")

In [38]:
!ls -l student-hw-omo.parquet

total 4
-rw-r--r-- 1 root root 1114 Jan 21 17:14 part-00000-ff89afda-afff-4aed-a50d-c2ee2460fe1d-c000.snappy.parquet
-rw-r--r-- 1 root root    0 Jan 21 17:14 _SUCCESS


In [39]:
df_new = spark.read.parquet("student-hw-omo.parquet/")

In [40]:
df_new.show()

+--------+-------------+-------------+
| student|height inches|weight pounds|
+--------+-------------+-------------+
|student6|           62|           85|
|student3|           69|           95|
|student7|           65|           80|
|student5|           60|           80|
|student2|           59|          100|
|student1|           64|           90|
+--------+-------------+-------------+



In [41]:
from pyspark.sql.functions import expr

Applying Transformation to the columns

In [51]:
df = df_new.withColumnRenamed("height inches", "height_inches")
df = df.withColumnRenamed("weight pounds", "weight_pounds")


df.show()

+--------+-------------+-------------+
| student|height_inches|weight_pounds|
+--------+-------------+-------------+
|student6|           62|           85|
|student3|           69|           95|
|student7|           65|           80|
|student5|           60|           80|
|student2|           59|          100|
|student1|           64|           90|
+--------+-------------+-------------+



In [52]:
df = df.withColumn("height_cm", expr("height_inches *2.4"))
df.show()

+--------+-------------+-------------+---------+
| student|height_inches|weight_pounds|height_cm|
+--------+-------------+-------------+---------+
|student6|           62|           85|    148.8|
|student3|           69|           95|    165.6|
|student7|           65|           80|    156.0|
|student5|           60|           80|    144.0|
|student2|           59|          100|    141.6|
|student1|           64|           90|    153.6|
+--------+-------------+-------------+---------+



In [53]:
df = df.withColumn("weight_kg", expr("weight_pounds *0.453592"))
df.show()

+--------+-------------+-------------+---------+---------+
| student|height_inches|weight_pounds|height_cm|weight_kg|
+--------+-------------+-------------+---------+---------+
|student6|           62|           85|    148.8|38.555320|
|student3|           69|           95|    165.6|43.091240|
|student7|           65|           80|    156.0|36.287360|
|student5|           60|           80|    144.0|36.287360|
|student2|           59|          100|    141.6|45.359200|
|student1|           64|           90|    153.6|40.823280|
+--------+-------------+-------------+---------+---------+



In [54]:
df.write.mode("overwrite").csv("student_transformed.csv",header=True)

In [59]:
!rm -rf student-hw.parquet
!ls

sample_data  student-hw.csv  student-hw-omo.parquet  student_transformed.csv


In [60]:
df = spark.read.csv("student_transformed.csv",header= True, inferSchema=True)

In [61]:
spark.stop()

In [62]:

spark= SparkSession.builder.appName("Exercises - ETL using Spark").getOrCreate()

In [63]:
df= spark.read.csv("student_transformed.csv",header=True,inferSchema=True)

In [64]:
df.show()

+--------+-------------+-------------+---------+---------+
| student|height_inches|weight_pounds|height_cm|weight_kg|
+--------+-------------+-------------+---------+---------+
|student6|           62|           85|    148.8| 38.55532|
|student3|           69|           95|    165.6| 43.09124|
|student7|           65|           80|    156.0| 36.28736|
|student5|           60|           80|    144.0| 36.28736|
|student2|           59|          100|    141.6|  45.3592|
|student1|           64|           90|    153.6| 40.82328|
+--------+-------------+-------------+---------+---------+



In [66]:
df = df.withColumn("height_meters", expr("height_cm /100"))
df.show()

+--------+-------------+-------------+---------+---------+------------------+
| student|height_inches|weight_pounds|height_cm|weight_kg|     height_meters|
+--------+-------------+-------------+---------+---------+------------------+
|student6|           62|           85|    148.8| 38.55532|1.4880000000000002|
|student3|           69|           95|    165.6| 43.09124|             1.656|
|student7|           65|           80|    156.0| 36.28736|              1.56|
|student5|           60|           80|    144.0| 36.28736|              1.44|
|student2|           59|          100|    141.6|  45.3592|             1.416|
|student1|           64|           90|    153.6| 40.82328|             1.536|
+--------+-------------+-------------+---------+---------+------------------+



In [67]:
df = df.withColumn("bmi", expr("weight_kg /(height_meters/weight_kg)"))
df.show()

+--------+-------------+-------------+---------+---------+------------------+------------------+
| student|height_inches|weight_pounds|height_cm|weight_kg|     height_meters|               bmi|
+--------+-------------+-------------+---------+---------+------------------+------------------+
|student6|           62|           85|    148.8| 38.55532|1.4880000000000002| 999.0004706333334|
|student3|           69|           95|    165.6| 43.09124|             1.656|1121.2892299140096|
|student7|           65|           80|    156.0| 36.28736|              1.56|  844.084933185641|
|student5|           60|           80|    144.0| 36.28736|              1.44| 914.4253442844445|
|student2|           59|          100|    141.6|  45.3592|             1.416|1453.0063733333334|
|student1|           64|           90|    153.6| 40.82328|             1.536|1084.9871028374998|
+--------+-------------+-------------+---------+---------+------------------+------------------+



In [75]:
df = df.drop("height_cm","weight_kg","height_meters")
df.show()

+--------+-------------+-------------+------------------+
| student|height_inches|weight_pounds|               bmi|
+--------+-------------+-------------+------------------+
|student6|           62|           85| 999.0004706333334|
|student3|           69|           95|1121.2892299140096|
|student7|           65|           80|  844.084933185641|
|student5|           60|           80| 914.4253442844445|
|student2|           59|          100|1453.0063733333334|
|student1|           64|           90|1084.9871028374998|
+--------+-------------+-------------+------------------+



In [76]:
from pyspark.sql.functions import col, round

df = df.withColumn("bmi_rounded", round(col("bmi")))
df.show()

+--------+-------------+-------------+------------------+-----------+
| student|height_inches|weight_pounds|               bmi|bmi_rounded|
+--------+-------------+-------------+------------------+-----------+
|student6|           62|           85| 999.0004706333334|      999.0|
|student3|           69|           95|1121.2892299140096|     1121.0|
|student7|           65|           80|  844.084933185641|      844.0|
|student5|           60|           80| 914.4253442844445|      914.0|
|student2|           59|          100|1453.0063733333334|     1453.0|
|student1|           64|           90|1084.9871028374998|     1085.0|
+--------+-------------+-------------+------------------+-----------+



In [78]:
df= df.repartition(1)

In [79]:
df.write.mode("overwrite").parquet("student_Jazzup.parquet")

In [80]:
!ls

sample_data	student-hw-omo.parquet	student_transformed.csv
student-hw.csv	student_Jazzup.parquet


In [81]:
spark.stop()